This code is run and tested on the Python 3.7.4 interpreter. Owlready2 was installed with `pip install owlready2`. It is the latest version owlready2 at this time - 0.40

Note that aliasing owlready2 as owl is not a common practice (like aliasing numpy with np, or pandas with pd) - I just like it.

In [2]:
import owlready2 as owl 
owl.VERSION

'0.40'

If we wanted to access an ontology we need to load it.

In [3]:
ontology = owl.get_ontology('../../ontology/io_prototype.rdf').load()

Access to the lists of entries within the ontology is implemented with the generators

In [4]:
print(ontology.classes())
print(ontology.annotation_properties())
print(ontology.data_properties())
print(ontology.object_properties())
print(ontology.individuals())

<generator object _GraphManager.classes at 0x7f4059c97ed0>
<generator object _GraphManager.annotation_properties at 0x7f4059c97ed0>
<generator object _GraphManager.data_properties at 0x7f4059c97ed0>
<generator object _GraphManager.object_properties at 0x7f4059c97ed0>
<generator object _GraphManager.individuals at 0x7f4059c97ed0>


In [5]:
for individual in ontology.individuals():
    print(individual)

io_prototype.mass
io_prototype.volume
io_prototype.boiled_carrots
io_prototype.carrot_raw_diced
io_prototype.drinking_water
io_prototype.x_idi1
io_prototype.x_idi2
io_prototype.x_ings1
io_prototype.x_ingsp1
io_prototype.x_ingsp2


* Owlready2 * Warning: ignoring cyclic type of, involving storid 2309

* Owlready2 * Warning: ignoring cyclic type of, involving storid 2372



Note that most of the classes in the current prototype are imported from OBO, FoodON, UO, PATO and general ontologies, such as IAO or RO.

In [6]:
for cls in ontology.classes():
    print(cls, end=', ')

obo.BFO_0000002, obo.BFO_0000003, obo.BFO_0000004, obo.BFO_0000017, obo.BFO_0000019, obo.BFO_0000020, obo.BFO_0000031, obo.BFO_0000015, obo.BFO_0000023, io_prototype.food_reciepe, obo.BFO_0000001, oboInOwl.ObsoleteClass, obo.BFO_0000008, obo.BFO_0000011, obo.BFO_0000035, obo.BFO_0000016, obo.BFO_0000034, obo.BFO_0000040, obo.BFO_0000141, obo.CHEBI_24432, obo.CHEBI_50906, obo.CHEBI_33290, obo.CHEBI_52211, obo.FOODON_00002403, obo.GO_0003674, obo.GO_0005575, obo.GO_0008150, obo.OBI_0000011, obo.IAO_0000003, obo.IAO_0000009, obo.IAO_0000005, obo.IAO_0000033, obo.IAO_0000007, obo.IAO_0000030, obo.IAO_0000027, obo.IAO_0000028, obo.IAO_0000100, obo.IAO_0000104, obo.OBI_0000047, obo.OBI_0000086, obo.OBI_0000094, obo.OBI_0000456, obo.OBI_0600010, obo.OBI_0000968, obo.OBI_0001755, obo.OBI_0001928, obo.OBI_0600008, obo.PATO_0000001, obo.PATO_0000004, obo.PATO_0001018, obo.PATO_0000008, obo.PATO_0001906, obo.PATO_0000011, obo.PATO_0000165, obo.PATO_0000014, obo.PATO_0000051, obo.PATO_0001300, obo

If we know the name of the class we want to process (it's actual name within the namespace of the ontology, not the label), we can get it like this:

In [7]:
ontology.food_reciepe

io_prototype.food_reciepe

However, generally we will be dealing with imported classes. Loading a class, knowing it's IRI is also quite simple.

In [8]:
owl.IRIS['http://purl.obolibrary.org/obo/BFO_0000002']

obo.BFO_0000002

In the current version I'm using my own definitions for the following classes:

 - food reciepe
 - device set
 - ingredient set
 - instruction set
 - ingredient specification
and properties
 - is about
 - has member
 - has quantity


However, all of them have equivalents in the source ontologies

 - Ingredient set: http://purl.obolibrary.org/obo/FOODON_00004082
 - Instruction set: http://purl.obolibrary.org/obo/FOODON_00004084
 - Device set: http://purl.obolibrary.org/obo/FOODON_00004084
 - Food reciepe: http://purl.obolibrary.org/obo/FOODON_00004081
 - Ingredient specification: http://purl.obolibrary.org/obo/FOODON_00004085

object properties
- 'is about' : http://purl.obolibrary.org/obo/IAO_0000136
- 'has member' : http://purl.obolibrary.org/obo/RO_0002351

data properties:
'has quantity' : http://purl.obolibrary.org/obo/COB_0000511

we will very likely end up using the existing properties.

Currently in the ontology, there is one food reciepe: Boiled Carrots

In [9]:
food_reciepe_class : owl.ClassConstruct = ontology.food_reciepe
for instance in ontology.get_instances_of(food_reciepe_class):
    print(instance)

io_prototype.boiled_carrots


Currently we are using a one-directional component_of relation. So we cannot get the ingredient set directly from this individual. Instead we need to search trough all ingredient sets and check wether it is connected to this reciepe via 'component_of' relation

In [36]:
food_reciepe_class : owl.ClassConstruct = ontology.food_reciepe
component_class : owl.ClassConstruct = ontology.ingredient_set
for instance in ontology.get_instances_of(food_reciepe_class):
    print(instance)
    for component_set in ontology.get_instances_of(component_class):
        if instance in component_set.component_of:
            print(component_set)
        

io_prototype.boiled_carrots
io_prototype.x_ings1


Specific components of the component set are connected to the set via 'has_component' relation.

In [41]:
food_reciepe_class : owl.ClassConstruct = ontology.food_reciepe
component_class : owl.ClassConstruct = ontology.ingredient_set
for instance in ontology.get_instances_of(food_reciepe_class):
    print(instance)
    for component_set in ontology.get_instances_of(component_class):
        if instance in component_set.component_of:
            print(f'\t{component_set}')
            for component in component_set.has_component:
                    print(f'\t\t{component}')

io_prototype.boiled_carrots
	io_prototype.x_ings1
		io_prototype.x_ingsp1
		io_prototype.x_ingsp2


Components are connected with their specification via 'has_component' relation. Components are about food materials. The relation between food material and components is 'is_about'.

In [62]:
food_reciepe_class : owl.ClassConstruct = ontology.food_reciepe
component_class : owl.ClassConstruct = ontology.ingredient_set
for instance in ontology.get_instances_of(food_reciepe_class):
    print(instance)
    for component_set in ontology.get_instances_of(component_class):
        if instance in component_set.component_of:
            print(f'\t{component_set}')
            for component in component_set.has_component:
                    print(f'\t\t{component}')
                    for food in component.is_about:
                        print(f'\t\t\tIs About: {food}')
                    for specification in component.has_component:
                        print(f'\t\t\t{specification}')
                        print(f'\t\t\t\t{specification.has_unit[0].label[0]}')
                        print(f'\t\t\t\t{specification.has_quantity[0]}')
                        print(f'\t\t\t\t{specification.is_quality_measure_of[0]}')

io_prototype.boiled_carrots
	io_prototype.x_ings1
		io_prototype.x_ingsp1
			Is About: io_prototype.drinking_water
			io_prototype.x_idi1
				teaspoon
				0.5
				io_prototype.volume
		io_prototype.x_ingsp2
			Is About: io_prototype.carrot_raw_diced


In [24]:
owl.IRIS['http://purl.obolibrary.org/obo/PATO_0002006'].label

['2-D shape']